In [47]:
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import date, datetime, timedelta

In [49]:
# Opening saved model
with open("model/classifier.pkl", "rb") as file:
    classifier = pickle.load(file)


In [50]:
product_data = pd.read_csv("dataset/Project Dataset/product.csv")

In [51]:
product_data = product_data.sort_values("product").reset_index().drop(columns=['index'])
product_data

,product,id,price
0,12 C AC,101,67300
1,12 inch Fan,57,3990
2,14 inch Fan,56,4490
3,17 inch Fan,54,6290
4,18 A AC,99,49900
...,...,...,...
118,Tornedo Fan,66,1090
119,Wall Move,68,3130
120,Washing Machine,84,19900
121,Water Filter,69,3900


In [52]:
today = date.today()
prediction_dates = [today + timedelta(days = i) for i in range(30)]

sql = "INSERT INTO `predictions` (`id`, `date`, `product_id`, `unit`, `user_id`, `created_at`, `updated_at`) VALUES "

for date in prediction_dates:
    result = classifier.predict(pd.DataFrame({'day_of_year': date.timetuple().tm_yday, 'month':date.month, 'day_of_week':date.weekday()}, index = [0]))
    i = 0
    for value in result[0]:
        if not value == 0:
            sql += "(NULL, '" + str(date) + "', '" + str(product_data['id'][i]) + "', '" + str(value) + "', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),"
        i += 1
print(sql)

INSERT INTO `predictions` (`id`, `date`, `product_id`, `unit`, `user_id`, `created_at`, `updated_at`) VALUES (NULL, '2024-09-07', '54', '2', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '141', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '127', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '88', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '83', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '81', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '67', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '56', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '153', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '87', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '133', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '88', '1', '5', CURRENT_TIMESTAMP, CURREN